# Docker Cookbook

**Eric Kerfoot\
School of Biomedical Engineering & Imaging Sciences\
King's College London**

This is a series of Dockerfile examples and command lines designed to illustrate some concepts and patterns in using Docker. All the examples are presented here as runnable notebook cells you can try without resorting to a terminal.

### Hello, World
Run the simple Hello World example image:

In [1]:
!docker run -it --rm hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



### CUDA Example
Run a CUDA container with devices 0 and 1 selected. Note the argument for `--gpus` includes escaped double quote characters so that these are included in the string.

In [4]:
!docker run -it --rm --gpus \"device=0,1\" nvidia/cuda:10.1-cudnn7-runtime-ubuntu18.04 nvidia-smi

Wed Jun 10 22:25:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 980     Off  | 00000000:01:00.0  On |                  N/A |
| 12%   49C    P8    21W / 195W |    181MiB /  4036MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 00000000:02:00.0 Off |                  N/A |
| 53%   83C    P2   165W / 250W |  10619MiB / 12196MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
        

### Creating an Internal User Account

The command of a container normally runs under root, this is not necessarily secure and files written to a volume are going to be owned by root. Using an internal unprivileged user account solves this issue but files generated in mount directory are owned by `root`. An alternative solution to using the `--user` flag is to use Linux commands to create a local unprivileged user account with the same UID and GID of the user launching the container. Since UID/GID are the important identifiers for an account, files written to the host file system will be owned by the launching user. The UID and GID can be passed to a Dockerfile through command line arguments to set these values internally, and the `USER` command causes all subsequent operations to be performed using this account. 

In [22]:
%%bash

# create a temporary directory
TMP_DIR=$(mktemp -d)

# write the following, up to EOF, to Dockerfile in the temp directory
cat - <<EOF > $TMP_DIR/Dockerfile
FROM alpine:3.12
    
ARG USER_ID
ARG GROUP_ID

RUN addgroup -g \${GROUP_ID} -S usergroup 
RUN adduser -G usergroup -D -S -u \${USER_ID} localuser

USER localuser
WORKDIR /home/localuser

CMD ["id"]
EOF

# Build the image, passing in the user ID from the environment variable UID and the group ID from the command "id -g"
# the effect of this is that the user "localuser" in the image will have the same UID and GID as your user, so the
# files it creates in volumes mounted from directories on your host system will be owned by you rather than root.
docker build --build-arg USER_ID=${UID} --build-arg GROUP_ID=$(id -g) -t localuser-test $TMP_DIR

# this will print out the ID information for the current user, which should be the created user and not root
echo
echo "Docker user info:"
docker run --rm localuser-test
echo

# remove image
docker image rm localuser-test


Step 1/8 : FROM alpine:3.12
 ---> a24bb4013296
Step 2/8 : ARG USER_ID
 ---> Using cache
 ---> 444f6a8d317d
Step 3/8 : ARG GROUP_ID
 ---> Using cache
 ---> 792bbb1cefab
Step 4/8 : RUN addgroup -g ${GROUP_ID} -S usergroup
 ---> Running in 361b4098e5ec
Removing intermediate container 361b4098e5ec
 ---> b53fe1308375
Step 5/8 : RUN adduser -G usergroup -D -S -u ${USER_ID} localuser
 ---> Running in a355346da620
Removing intermediate container a355346da620
 ---> 656ec8617d5d
Step 6/8 : USER localuser
 ---> Running in 37466f03f083
Removing intermediate container 37466f03f083
 ---> f2f8dd27c20a
Step 7/8 : WORKDIR /home/localuser
 ---> Running in b898c3594339
Removing intermediate container b898c3594339
 ---> 786ac25c3258
Step 8/8 : CMD ["id"]
 ---> Running in 4c25a7c9cb89
Removing intermediate container 4c25a7c9cb89
 ---> a67447c01a1c
Successfully built a67447c01a1c
Successfully tagged localuser-test:latest
uid=8867(localuser) gid=2008(usergroup) groups=2008(usergroup)
Untagged: localuser-tes

### Stateless Build

This shell script illustrates using stdin streams to send a Dockerfile line by line to the command rather than saving it as a file. This stateless script doesn't require local files, instead using the `sh/bash` "here-document" functionality to pass commands through stdin. The `<<EOF` shell command states that stdin will be passed all text up to the line containing the delimiter "EOF", `docker build` is told to read the Dockerfile from stdin with the `-` argument. Note when this is used Dockes does not create a build context, that is it won't copy the files in the build directory to a temporary location, so commands like `COPY` cannot be used.

In [5]:
%%sh

REPO=hello_stateless
TAG=latest

# build by passing in the Dockerfile as input on stdin rather than as a file
docker build -t $REPO:$TAG - <<EOF

FROM python:3.7
RUN echo "print(\"Hello, stateless\")\n" > hello.py
CMD ["python","hello.py"]

EOF

docker run --rm $REPO:$TAG
    
# remove the image immediately after running it 
docker image rm $REPO


Step 1/3 : from python:3.7
 ---> e497dabd8450
Step 2/3 : RUN echo "print(\"Hello, stateless\")\n" > hello.py
 ---> Running in bb24a0beced7
Removing intermediate container bb24a0beced7
 ---> d852aec8a79d
Step 3/3 : CMD ["python","hello.py"]
 ---> Running in b9c8075efaf1
Removing intermediate container b9c8075efaf1
 ---> 91a03ace18f5
Successfully built 91a03ace18f5
Successfully tagged hello_stateless:latest
Hello, stateless
Untagged: hello_stateless:latest
Deleted: sha256:91a03ace18f5f79dae591df0ab544424b0bc3ef90e54ed6da2a91c14117dae9f
Deleted: sha256:d852aec8a79d712d5cb386081b0708cb0c004699b27f6133b7bede7dbbc26901
Deleted: sha256:d1e389fd6a5016de531c6777c15d7e23753855f95ae314ab5378b990c518444c


### Save and Load Compressed

This illustrates saving a docker file to an aggressively compressed tarball then loading it again using the `save` and `load` commands. The commands below use `gzip/gunzip` to compress and decompress the tarball used by Docker. It's not necessary to use these tools but instead save to an uncompressed .tar file and load that file directly.

In [14]:
%%sh

# save the image to a tarball, passing through gzip to compress it
docker image save hello-flask | gzip -9 > hello_flask.tgz
ls -lh *.tgz

# decompress tarball and pipe to docker
gunzip -c hello_flask.tgz | docker load

-rw-r--r-- 1 localek10 bioeng 330M Jun 10 23:34 hello_flask.tgz
Loaded image: hello-flask:latest


### Starting and Controlling a Detached Container

This series of commands starts a new container in a detached state (`-d`), so running in the background. Docker returns the container ID when this is done so that it can be captured in a script variable and used later as input to the other commands. An example of `docker container exec` is provided here to demonstrate invoking a command within a running container without the need to attach a terminal to it for direct interaction.

In [15]:
%%sh
# create a container and capture the container ID
CON_ID=$(docker run -d -p 5000:5000 hello-flask)
echo Container ID: $CON_ID

# view app output
sleep 1
echo Output: $(curl -s localhost:5000)

# list containers
echo
docker container ls

# run `ls -l` in the container
echo
docker container exec $CON_ID ls -l

# whack the container using the ID
echo
docker container rm --force $CON_ID

Container ID: f217e82a874293e9e3cfd309f8410aff2f6d01c9a75671924a39106e9e7acedf
Output: Hello, World!

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
f217e82a8742        hello-flask         "flask run --host=0.…"   2 seconds ago       Up 1 second         0.0.0.0:5000->5000/tcp   keen_elgamal

total 72
drwxr-xr-x   2 root root 4096 Jun 10 22:41 __pycache__
drwxr-xr-x   1 root root 4096 May 15 17:33 bin
drwxr-xr-x   2 root root 4096 May  2 16:39 boot
drwxr-xr-x   5 root root  340 Jun 10 22:41 dev
drwxr-xr-x   1 root root 4096 Jun 10 22:41 etc
-rw-r--r--   1 root root  110 May 27 19:47 hello_flask.py
drwxr-xr-x   2 root root 4096 May  2 16:39 home
drwxr-xr-x   1 root root 4096 May 15 17:33 lib
drwxr-xr-x   2 root root 4096 May 14 14:50 lib64
drwxr-xr-x   2 root root 4096 May 14 14:50 media
drwxr-xr-x   2 root root 4096 May 14 14:50 mnt
drwxr-xr-x   2 root root 4096 May 14 14:50 opt
dr-xr-xr-x 311 root 

### Reading and Writing Streams

When a container is run it can read from stdin whatever is streamed to the `docker run` command. Using pipes (`|`) and stream redirection (`<`, `>`), files or inline data can be streamed to the command being run in the container. This is one way of passing data into a container without having to mount volumes.

In [36]:
%%sh

REPO=stdin_test

# build by passing in the Dockerfile as input on stdin rather than as a file
docker build -t $REPO - <<EOF

FROM alpine:3.12
CMD ["/bin/sh","-c","read i; echo Received: \$i"]

EOF

# run the container with the given string written to its stdin stream
echo "Hello, pipe" | docker run -i --rm $REPO

# save text to a temporary file
echo "Hello, stdin" > input.txt

# run container streaming input.txt to stdin and stdout to output.txt
docker run -i --rm $REPO < input.txt > output.txt
    
echo
echo "Output from container:"
cat output.txt

# run again with stdin read from the here-document up to EOF
docker run -i --rm $REPO <<EOF > output.txt
Hello streamed document
EOF

echo
echo "Output from container:"
cat output.txt

# remove the image and temporary files
docker image rm $REPO
rm input.txt output.txt


Step 1/2 : FROM alpine:3.12
 ---> a24bb4013296
Step 2/2 : CMD ["/bin/sh","-c","read i; echo Received: $i"]
 ---> Running in 8a8fa4d4618e
Removing intermediate container 8a8fa4d4618e
 ---> b821aab04590
Successfully built b821aab04590
Successfully tagged stdin_test:latest
Received: Hello, pipe

Output from container:
Received: Hello, stdin

Output from container:
Received: Hello streamed document
Untagged: stdin_test:latest
Deleted: sha256:b821aab04590e91a0f12a1a2e2dcce7bb38a5b479428eba73a321a075d2b9bcd


### ENTRYPOINT vs. CMD

`ENTRYPOINT` is used to define a command which will always be run when a container runs, values passed to `CMD` or on the command line are included as further arguments to this command. `CMD` typically is used to also define the command to run, but this can be overridden on the command line.

The following script creates an image which runs `echo` as its command with no further arguments. Running the image with no configuration prints an empty line, providing further arguments replaces the call to `echo` with whatever was provided. An entrypoint to modify this behaviour can be explicitly set in the command or in the Dockerfile.

In [37]:
%%sh

REPO=entrypoint_test

# define an image with just a CMD statement
docker build -t $REPO - <<EOF
from alpine:3.12
CMD ["echo"]
EOF

# should print ""
docker run --rm $REPO

# should print a hostname string by running the command "hostname" instead of echo
docker run --rm $REPO hostname

# explicitly set the entrypoint to be echo, now prints "hostname" rather than running that command
docker run --rm --entrypoint echo $REPO hostname

# now we set the entrypoint in the Dockerfile itself
docker build -t $REPO - <<EOF
from alpine:3.12
ENTRYPOINT ["echo"]
CMD ["Default", "arguments"]
EOF

# should print "Default arguments"
docker run --rm $REPO

# should print "Hello CMD"
docker run --rm $REPO Hello CMD

# remove the image and temporary files
docker image rm $REPO


Step 1/2 : from alpine:3.12
 ---> a24bb4013296
Step 2/2 : CMD ["echo"]
 ---> Running in 70c8353f55bc
Removing intermediate container 70c8353f55bc
 ---> 821dd441a8b4
Successfully built 821dd441a8b4
Successfully tagged entrypoint_test:latest

9115b92e6a86
hostname

Step 1/3 : from alpine:3.12
 ---> a24bb4013296
Step 2/3 : ENTRYPOINT ["echo"]
 ---> Running in 777379b13b1b
Removing intermediate container 777379b13b1b
 ---> 75d46e6ea330
Step 3/3 : CMD ["Default", "arguments"]
 ---> Running in 57c6cd256ee9
Removing intermediate container 57c6cd256ee9
 ---> 090569747dac
Successfully built 090569747dac
Successfully tagged entrypoint_test:latest
Default arguments
Hello CMD
Untagged: entrypoint_test:latest
Deleted: sha256:090569747dac608b183ef926487c6e8c3ed850688a2b766a6163289b2e442f38
Deleted: sha256:75d46e6ea330c6f7205d5fa11acd5e38488b2d08a751dfd0bb046d5b6bb903d7


### Docker GUI with X

GUI programs can be run through Docker which will connect to the host's X server to display the interface. Libraries for X need to be installed as well as extra rendering components like fonts which won't be present on stripped-down images like Alpine. The files for building this image are in `./hello_gui`:

In [48]:
%%sh 

cd hello_gui

echo "Python File:"
cat hello.py
echo "========="
echo "Dockerfile:"
cat Dockerfile
echo "========="

REPO=hello-qt

docker build -t $REPO .

Python File:
import sys, platform
from PyQt5 import QtWidgets

if __name__ == '__main__':
   app = QtWidgets.QApplication(sys.argv)
   w = QtWidgets.QWidget()
   b = QtWidgets.QLabel(w)
   b.setText(f"Hello World! (Python {platform.python_version()})")
   w.setGeometry(100,100,200,50)
   b.move(30,20)
   w.setWindowTitle("PyQt in Docker")
   w.show()
   sys.exit(app.exec_())

Dockerfile:
FROM alpine:3.12

RUN apk update
RUN apk add py3-qt5 ttf-ubuntu-font-family

COPY hello.py /

CMD ["python3","hello.py"]

Step 1/5 : FROM alpine:3.12
 ---> a24bb4013296
Step 2/5 : RUN apk update
 ---> Using cache
 ---> fb6880ab223a
Step 3/5 : RUN apk add py3-qt5 ttf-ubuntu-font-family
 ---> Using cache
 ---> 5a21447c49f4
Step 4/5 : COPY hello.py /
 ---> Using cache
 ---> e9965649a613
Step 5/5 : CMD ["python3","hello.py"]
 ---> Using cache
 ---> 7b02a6d0c47d
Successfully built 7b02a6d0c47d
Successfully tagged hello-qt:latest


With the image built the command line to run must allow X ports to be passed through (or use `--net=host`), the `DISPLAY` variable to be set so that the program know which display to render to, and the X credentials must be passed in for the current user. The following command will need to be run within the environment hosting the X server:

In [ ]:
%%sh

docker run -ti --rm --net=host -e DISPLAY="$DISPLAY" --volume="$HOME/.Xauthority:/root/.Xauthority:rw" hello-qt